### Student Information
Name: 蕭泓佐

Student ID: r13922163

GitHub ID: davidshiao55

Kaggle name: David Shiao

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

# Data

In [3]:
# !mkdir -p ~/.kaggle
# !cp /DM2024-Lab2-Homework/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c dm-2024-isa-5810-lab-2-homework
# !unzip -q dm-2024-isa-5810-lab-2-homework.zip

dm-2024-isa-5810-lab-2-homework.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data_identification.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
import pandas as pd

data_ids = pd.read_csv('data_identification.csv')
data_ids.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [9]:
train_data_ids = data_ids[data_ids['identification'] == 'train'].drop(['identification'], axis=1)
test_data_ids = data_ids[data_ids['identification'] == 'test'].drop(['identification'], axis=1)

train_data_ids.head()

,tweet_id
1,0x29e452
2,0x2b3819
4,0x2a2acc
5,0x2a8830
6,0x20b21d


In [10]:
emotion_labels = pd.read_csv('emotion.csv')
emotion_labels.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [11]:
tweets_df = pd.read_json('tweets_DM.json', lines=True)

# Use json_normalize to flatten only the `_source` column and keep other columns intact
source_df = pd.json_normalize(tweets_df['_source'])

# Concatenate the flattened `_source` columns with the original dataframe
tweets_df = pd.concat([tweets_df.drop(columns=['_source']), source_df], axis=1)

tweets_df.head()

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [43]:
train_data = pd.merge(train_data_ids, emotion_labels, on='tweet_id', how='inner')
train_data = pd.merge(train_data, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
train_data.head()

,tweet_id,emotion,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x29e452,joy,809,hashtag_tweets,2015-01-17 03:07:03,tweets,[],0x29e452,Huge Respect🖒 @JohnnyVegasReal talking about l...
1,0x2b3819,joy,808,hashtag_tweets,2016-07-02 09:34:06,tweets,"[spateradio, app]",0x2b3819,Yoooo we hit all our monthly goals with the ne...
2,0x2a2acc,trust,16,hashtag_tweets,2016-08-15 18:18:39,tweets,[],0x2a2acc,@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...
3,0x2a8830,joy,768,hashtag_tweets,2017-02-11 08:49:46,tweets,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",0x2a8830,Come join @ambushman27 on #PUBG while he striv...
4,0x20b21d,anticipation,70,hashtag_tweets,2016-11-23 05:37:10,tweets,"[strength, bones, God]",0x20b21d,@fanshixieen2014 Blessings!My #strength little...


In [13]:
test_data = pd.merge(test_data_ids, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
test_data.head()

,tweet_id,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x28cc61,107,hashtag_tweets,2017-01-17 14:13:32,tweets,[],0x28cc61,@Habbo I've seen two separate colours of the e...
1,0x2db41f,728,hashtag_tweets,2015-10-17 06:46:20,tweets,[],0x2db41f,@FoxNews @KellyannePolls No serious self respe...
2,0x2466f6,491,hashtag_tweets,2016-12-19 03:50:27,tweets,[womendrivers],0x2466f6,"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,28,hashtag_tweets,2017-04-09 19:32:19,tweets,[robbingmembers],0x23f9e9,@cineworld “only the brave” just out and fount...
4,0x1fb4e1,925,hashtag_tweets,2016-01-15 11:59:31,tweets,[],0x1fb4e1,Felt like total dog 💩 going into open gym and ...


# Data preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

X = train_data['tweet.text']
y = train_data['emotion']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [28]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [40]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

label_encoder = LabelEncoder()
y_train_label = label_encoder.fit_transform(y_train)
y_val_label = label_encoder.transform(y_val)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
X_train_encoding = tokenizer(list(X_train), truncation=True, padding=True, max_length=128, return_tensors="pt")
X_val_encoding = tokenizer(list(X_val), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [ ]:
from torch.utils.data import DataLoader, Dataset

# Dataset class for loading text data from DataFrame
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # Should already contain tensors if return_tensors="pt" was used
        self.labels = torch.tensor(labels)  # Ensure labels are converted to a tensor

    def __getitem__(self, idx):
        # Use encodings as they are without re-wrapping in torch.tensor
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Initialize datasets
train_dataset = TweetDataset(X_train_encoding, y_train_label)
val_dataset = TweetDataset(X_val_encoding, y_val_label)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [ ]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm  

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8).to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

# Training loop
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    model.train()
    total_train_loss = 0

    # Wrap the training DataLoader with tqdm for progress tracking
    for batch in tqdm(train_dataloader, desc="Training"):
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        
        # Backward pass
        loss.backward()
        
        # Clip the gradient to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Update parameters
        optimizer.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    correct_predictions = 0
    total_predictions = 0

    # Wrap the validation DataLoader with tqdm for progress tracking
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validating"):
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)
            
            # Forward pass for validation
            outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            
            total_val_loss += loss.item()
            
            # Calculate accuracy
            _, preds = torch.max(logits, dim=1)
            correct_predictions += (preds == b_labels).sum().item()
            total_predictions += b_labels.size(0)
    
    avg_val_loss = total_val_loss / len(val_dataloader)
    accuracy = correct_predictions / total_predictions
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}, Accuracy: {accuracy}")

print("Training complete!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/3


Training: 100%|██████████| 375/375 [00:38<00:00,  9.77it/s]


Epoch 1/3, Training Loss: 1.702408638159434


Validating: 100%|██████████| 38/38 [00:01<00:00, 33.81it/s]


...

Epoch 2/3, Validation Loss: 1.5219614223430031, Accuracy: 0.4483333333333333

Epoch 3/3


Training: 100%|██████████| 375/375 [00:37<00:00,  9.93it/s]


Epoch 3/3, Training Loss: 1.0531256275177001


Validating: 100%|██████████| 38/38 [00:01<00:00, 34.15it/s]

Epoch 3/3, Validation Loss: 1.5581969399201243, Accuracy: 0.4683333333333333
Training complete!


In [ ]:
output_dir = "./models/fine_tuned_bert"
model.save_pretrained(output_dir)